In [4]:
import os
import sys
import time
import random
import string
import argparse

import torch
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data
import numpy as np

from utils import CTCLabelConverter, CTCLabelConverterForBaiduWarpctc, AttnLabelConverter, Averager
from dataset import hierarchical_dataset, AlignCollate, Batch_Balanced_Dataset
from model import Model
from test import validation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
def train(opt):
    """ dataset preparation """
    if not opt.data_filtering_off:
        print('Filtering the images containing characters which are not in opt.character')
        print('Filtering the images whose label is longer than opt.batch_max_length')
        # see https://github.com/clovaai/deep-text-recognition-benchmark/blob/6593928855fb7abb999a99f428b3e4477d4ae356/dataset.py#L130

    opt.select_data = opt.select_data.split('-')
    opt.batch_ratio = opt.batch_ratio.split('-')
    train_dataset = Batch_Balanced_Dataset(opt)

    log = open(f'./saved_models/{opt.exp_name}/log_dataset.txt', 'a')
    AlignCollate_valid = AlignCollate(imgH=opt.imgH, imgW=opt.imgW, keep_ratio_with_pad=opt.PAD)
    valid_dataset, valid_dataset_log = hierarchical_dataset(root=opt.valid_data, opt=opt)
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=opt.batch_size,
        shuffle=True,  # 'True' to check training progress with validation function.
        num_workers=int(opt.workers),
        collate_fn=AlignCollate_valid, pin_memory=True)
    log.write(valid_dataset_log)
    print('-' * 80)
    log.write('-' * 80 + '\n')
    log.close()
    
    """ model configuration """
    if 'CTC' in opt.Prediction:
        if opt.baiduCTC:
            converter = CTCLabelConverterForBaiduWarpctc(opt.character)
        else:
            converter = CTCLabelConverter(opt.character)
    else:
        converter = AttnLabelConverter(opt.character)
    opt.num_class = len(converter.character)

    if opt.rgb:
        opt.input_channel = 3
    model = Model(opt)
    print('model input parameters', opt.imgH, opt.imgW, opt.num_fiducial, opt.input_channel, opt.output_channel,
          opt.hidden_size, opt.num_class, opt.batch_max_length, opt.Transformation, opt.FeatureExtraction,
          opt.SequenceModeling, opt.Prediction)

    # weight initialization
    for name, param in model.named_parameters():
        if 'localization_fc2' in name:
            print(f'Skip {name} as it is already initialized')
            continue
        try:
            if 'bias' in name:
                init.constant_(param, 0.0)
            elif 'weight' in name:
                init.kaiming_normal_(param)
        except Exception as e:  # for batchnorm.
            if 'weight' in name:
                param.data.fill_(1)
            continue

    # data parallel for multi-GPU
    model = torch.nn.DataParallel(model).to(device)
    model.train()
    if opt.saved_model != '':
        print(f'loading pretrained model from {opt.saved_model}')
        if opt.FT:
            model.load_state_dict(torch.load(opt.saved_model), strict=False)
        else:
            model.load_state_dict(torch.load(opt.saved_model))
    print("Model:")
    print(model)

    """ setup loss """
    if 'CTC' in opt.Prediction:
        if opt.baiduCTC:
            # need to install warpctc. see our guideline.
            from warpctc_pytorch import CTCLoss 
            criterion = CTCLoss()
        else:
            criterion = torch.nn.CTCLoss(zero_infinity=True).to(device)
    else:
        criterion = torch.nn.CrossEntropyLoss(ignore_index=0).to(device)  # ignore [GO] token = ignore index 0
    # loss averager
    loss_avg = Averager()

    # filter that only require gradient decent
    filtered_parameters = []
    params_num = []
    for p in filter(lambda p: p.requires_grad, model.parameters()):
        filtered_parameters.append(p)
        params_num.append(np.prod(p.size()))
    print('Trainable params num : ', sum(params_num))
    # [print(name, p.numel()) for name, p in filter(lambda p: p[1].requires_grad, model.named_parameters())]

    # setup optimizer
    if opt.adam:
        optimizer = optim.Adam(filtered_parameters, lr=opt.lr, betas=(opt.beta1, 0.999))
    else:
        optimizer = optim.Adadelta(filtered_parameters, lr=opt.lr, rho=opt.rho, eps=opt.eps)
    print("Optimizer:")
    print(optimizer)

    """ final options """
    # print(opt)
    with open(f'./saved_models/{opt.exp_name}/opt.txt', 'a') as opt_file:
        opt_log = '------------ Options -------------\n'
        args = vars(opt)
        for k, v in args.items():
            opt_log += f'{str(k)}: {str(v)}\n'
        opt_log += '---------------------------------------\n'
        print(opt_log)
        opt_file.write(opt_log)

    """ start training """
    start_iter = 0
    if opt.saved_model != '':
        try:
            start_iter = int(opt.saved_model.split('_')[-1].split('.')[0])
            print(f'continue to train, start_iter: {start_iter}')
        except:
            pass

    start_time = time.time()
    best_accuracy = -1
    best_norm_ED = -1
    iteration = start_iter

    while(True):
        # train part
        image_tensors, labels = train_dataset.get_batch()
        image = image_tensors.to(device)
        text, length = converter.encode(labels, batch_max_length=opt.batch_max_length)
        batch_size = image.size(0)

        if 'CTC' in opt.Prediction:
            preds = model(image, text)
            preds_size = torch.IntTensor([preds.size(1)] * batch_size)
            if opt.baiduCTC:
                preds = preds.permute(1, 0, 2)  # to use CTCLoss format
                cost = criterion(preds, text, preds_size, length) / batch_size
            else:
                preds = preds.log_softmax(2).permute(1, 0, 2)
                cost = criterion(preds, text, preds_size, length)

        else:
            preds = model(image, text[:, :-1])  # align with Attention.forward
            target = text[:, 1:]  # without [GO] Symbol
            cost = criterion(preds.view(-1, preds.shape[-1]), target.contiguous().view(-1))

        model.zero_grad()
        cost.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), opt.grad_clip)  # gradient clipping with 5 (Default)
        optimizer.step()

        loss_avg.add(cost)

        # validation part
        if (iteration + 1) % opt.valInterval == 0 or iteration == 0: # To see training progress, we also conduct validation when 'iteration == 0' 
            elapsed_time = time.time() - start_time
            # for log
            with open(f'./saved_models/{opt.exp_name}/log_train.txt', 'a') as log:
                model.eval()
                with torch.no_grad():
                    valid_loss, current_accuracy, current_norm_ED, preds, confidence_score, labels, infer_time, length_of_data = validation(
                        model, criterion, valid_loader, converter, opt)
                model.train()

                # training loss and validation loss
                loss_log = f'[{iteration+1}/{opt.num_iter}] Train loss: {loss_avg.val():0.5f}, Valid loss: {valid_loss:0.5f}, Elapsed_time: {elapsed_time:0.5f}'
                loss_avg.reset()

                current_model_log = f'{"Current_accuracy":17s}: {current_accuracy:0.3f}, {"Current_norm_ED":17s}: {current_norm_ED:0.2f}'

                # keep best accuracy model (on valid dataset)
                if current_accuracy > best_accuracy:
                    best_accuracy = current_accuracy
                    torch.save(model.state_dict(), f'./saved_models/{opt.exp_name}/best_accuracy.pth')
                if current_norm_ED > best_norm_ED:
                    best_norm_ED = current_norm_ED
                    torch.save(model.state_dict(), f'./saved_models/{opt.exp_name}/best_norm_ED.pth')
                best_model_log = f'{"Best_accuracy":17s}: {best_accuracy:0.3f}, {"Best_norm_ED":17s}: {best_norm_ED:0.2f}'

                loss_model_log = f'{loss_log}\n{current_model_log}\n{best_model_log}'
                print(loss_model_log)
                log.write(loss_model_log + '\n')

                # show some predicted results
                dashed_line = '-' * 80
                head = f'{"Ground Truth":25s} | {"Prediction":25s} | Confidence Score & T/F'
                predicted_result_log = f'{dashed_line}\n{head}\n{dashed_line}\n'
                for gt, pred, confidence in zip(labels[:5], preds[:5], confidence_score[:5]):
                    if 'Attn' in opt.Prediction:
                        gt = gt[:gt.find('[s]')]
                        pred = pred[:pred.find('[s]')]

                    predicted_result_log += f'{gt:25s} | {pred:25s} | {confidence:0.4f}\t{str(pred == gt)}\n'
                predicted_result_log += f'{dashed_line}'
                print(predicted_result_log)
                log.write(predicted_result_log + '\n')

        # save model per 1e+5 iter.
        if (iteration + 1) % 1e+5 == 0:
            torch.save(
                model.state_dict(), f'./saved_models/{opt.exp_name}/iter_{iteration+1}.pth')

        if (iteration + 1) == opt.num_iter:
            print('end the training')
#             sys.exit()
            break
        iteration += 1

In [6]:
# parser = argparse.ArgumentParser()
# parser.add_argument('--exp_name', help='Where to store logs and models')
# parser.add_argument('--train_data', required=True, help='path to training dataset')
# parser.add_argument('--valid_data', required=True, help='path to validation dataset')
# parser.add_argument('--manualSeed', type=int, default=1111, help='for random seed setting')
# parser.add_argument('--workers', type=int, help='number of data loading workers', default=4)
# parser.add_argument('--batch_size', type=int, default=192, help='input batch size')
# parser.add_argument('--num_iter', type=int, default=300000, help='number of iterations to train for')
# parser.add_argument('--valInterval', type=int, default=2000, help='Interval between each validation')
# parser.add_argument('--saved_model', default='', help="path to model to continue training")
# parser.add_argument('--FT', action='store_true', help='whether to do fine-tuning')
# parser.add_argument('--adam', action='store_true', help='Whether to use adam (default is Adadelta)')
# parser.add_argument('--lr', type=float, default=1, help='learning rate, default=1.0 for Adadelta')
# parser.add_argument('--beta1', type=float, default=0.9, help='beta1 for adam. default=0.9')
# parser.add_argument('--rho', type=float, default=0.95, help='decay rate rho for Adadelta. default=0.95')
# parser.add_argument('--eps', type=float, default=1e-8, help='eps for Adadelta. default=1e-8')
# parser.add_argument('--grad_clip', type=float, default=5, help='gradient clipping value. default=5')
# parser.add_argument('--baiduCTC', action='store_true', help='for data_filtering_off mode')
# """ Data processing """
# parser.add_argument('--select_data', type=str, default='MJ-ST',
#                     help='select training data (default is MJ-ST, which means MJ and ST used as training data)')
# parser.add_argument('--batch_ratio', type=str, default='0.5-0.5',
#                     help='assign ratio for each selected data in the batch')
# parser.add_argument('--total_data_usage_ratio', type=str, default='1.0',
#                     help='total data usage ratio, this ratio is multiplied to total number of data.')
# parser.add_argument('--batch_max_length', type=int, default=25, help='maximum-label-length')
# parser.add_argument('--imgH', type=int, default=32, help='the height of the input image')
# parser.add_argument('--imgW', type=int, default=100, help='the width of the input image')
# parser.add_argument('--rgb', action='store_true', help='use rgb input')
# parser.add_argument('--character', type=str,
#                     default='0123456789abcdefghijklmnopqrstuvwxyz', help='character label')
# parser.add_argument('--sensitive', action='store_true', help='for sensitive character mode')
# parser.add_argument('--PAD', action='store_true', help='whether to keep ratio then pad for image resize')
# parser.add_argument('--data_filtering_off', action='store_true', help='for data_filtering_off mode')
# """ Model Architecture """
# parser.add_argument('--Transformation', type=str, required=True, help='Transformation stage. None|TPS')
# parser.add_argument('--FeatureExtraction', type=str, required=True,
#                     help='FeatureExtraction stage. VGG|RCNN|ResNet')
# parser.add_argument('--SequenceModeling', type=str, required=True, help='SequenceModeling stage. None|BiLSTM')
# parser.add_argument('--Prediction', type=str, required=True, help='Prediction stage. CTC|Attn')
# parser.add_argument('--num_fiducial', type=int, default=20, help='number of fiducial points of TPS-STN')
# parser.add_argument('--input_channel', type=int, default=1,
#                     help='the number of input channel of Feature extractor')
# parser.add_argument('--output_channel', type=int, default=512,
#                     help='the number of output channel of Feature extractor')
# parser.add_argument('--hidden_size', type=int, default=256, help='the size of the LSTM hidden state')


# 1. None-VGG-BiLSTM-CTC.pth (Best)

In [7]:
args = argparse.Namespace(
    train_data = "../datasets/forTrainingData/train",
    valid_data = "../datasets/forTrainingData/val",
    select_data = "01-02-03",
    batch_ratio = "0.3-0.3-0.4",
    Transformation = "None",
    FeatureExtraction = "VGG",
    SequenceModeling = "BiLSTM",
    Prediction = "CTC",
    saved_model = "models/None-VGG-BiLSTM-CTC.pth",
    FT = True,
    manualSeed = 213,
    
    # Add arg for Training
    num_gpu = None,
    exp_name = None,
    workers = 0,
    batch_size = 192,
    data_filtering_off = None,
    imgH = 32,
    imgW = 100,
    num_iter = 5000,
    valInterval = 100, 
    adam = None,
    lr = 1,
    beta1 = 0.9,
    rho = 0.95,
    eps = 1e-8,
    grad_clip = 5,
    baiduCTC = None,
    total_data_usage_ratio = "1.0",
    batch_max_length = 25,
    rgb = None,
    sensitive = None,
    PAD = None,
    num_fiducial = 20,
    input_channel = 1,
    output_channel = 512,
    hidden_size = 256,
    character = '0123456789abcdefghijklmnopqrstuvwxyz'
    
)

In [8]:
opt = args

if not opt.exp_name:
    opt.exp_name = f'{opt.Transformation}-{opt.FeatureExtraction}-{opt.SequenceModeling}-{opt.Prediction}'
    opt.exp_name += f'-Seed{opt.manualSeed}'
    # print(opt.exp_name)

os.makedirs(f'./saved_models/{opt.exp_name}', exist_ok=True)

""" vocab / character number configuration """
if opt.sensitive:
    # opt.character += 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    opt.character = string.printable[:-6]  # same with ASTER setting (use 94 char).

""" Seed and GPU setting """
# print("Random Seed: ", opt.manualSeed)
random.seed(opt.manualSeed)
np.random.seed(opt.manualSeed)
torch.manual_seed(opt.manualSeed)
torch.cuda.manual_seed(opt.manualSeed)

cudnn.benchmark = True
cudnn.deterministic = True
opt.num_gpu = torch.cuda.device_count()
# print('device count', opt.num_gpu)
if opt.num_gpu > 1:
    print('------ Use multi-GPU setting ------')
    print('if you stuck too long time with multi-GPU setting, try to set --workers 0')
    # check multi-GPU issue https://github.com/clovaai/deep-text-recognition-benchmark/issues/1
    opt.workers = opt.workers * opt.num_gpu
    opt.batch_size = opt.batch_size * opt.num_gpu

    """ previous version
    print('To equlize batch stats to 1-GPU setting, the batch_size is multiplied with num_gpu and multiplied batch_size is ', opt.batch_size)
    opt.batch_size = opt.batch_size * opt.num_gpu
    print('To equalize the number of epochs to 1-GPU setting, num_iter is divided with num_gpu by default.')
    If you dont care about it, just commnet out these line.)
    opt.num_iter = int(opt.num_iter / opt.num_gpu)
    """

train(opt)

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: ../datasets/forTrainingData/train
opt.select_data: ['01', '02', '03']
opt.batch_ratio: ['0.3', '0.3', '0.4']
--------------------------------------------------------------------------------
dataset_root:    ../datasets/forTrainingData/train	 dataset: 01
sub-directory:	/01	 num samples: 2989
num total samples of 01: 2989 x 1.0 (total_data_usage_ratio) = 2989
num samples of 01 per batch: 192 x 0.3 (batch_ratio) = 58
--------------------------------------------------------------------------------
dataset_root:    ../datasets/forTrainingData/train	 dataset: 02
sub-directory:	/02	 num samples: 2872
num total samples of 02: 2872 x 1.0 (total_data_usage_ratio) = 2872
num samples of 02 per batch: 192 x 0.3 (batch_ratio) = 58
-----------------------------------

[400/5000] Train loss: 0.02390, Valid loss: 0.20420, Elapsed_time: 73.99482
Current_accuracy : 84.425, Current_norm_ED  : 0.97
Best_accuracy    : 84.425, Best_norm_ED     : 0.97
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
995b51j95                 | 995b51j95                 | 0.9574	True
9012f51r04                | 9012f51r04                | 0.9948	True
1001                      | 1001                      | 0.6322	True
z                         | 995c1741                  | 0.1647	False
3902                      | 3902                      | 0.7762	True
--------------------------------------------------------------------------------
[500/5000] Train loss: 0.01346, Valid loss: 0.19340, Elapsed_time: 90.34606
Current_accuracy : 85.762, Current_norm_ED  : 0.97
Best_accuracy    : 85.762, Best_

[1400/5000] Train loss: 0.00049, Valid loss: 0.27154, Elapsed_time: 236.57812
Current_accuracy : 85.829, Current_norm_ED  : 0.97
Best_accuracy    : 85.963, Best_norm_ED     : 0.97
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
5601                      | 5601                      | 0.4725	True
3401                      | 3401                      | 0.9503	True
995h13b08                 | 995h13b08                 | 0.7146	True
4401                      | 4401                      | 0.9228	True
2703                      | 2703                      | 0.9381	True
--------------------------------------------------------------------------------
[1500/5000] Train loss: 0.00044, Valid loss: 0.25072, Elapsed_time: 252.73040
Current_accuracy : 85.561, Current_norm_ED  : 0.97
Best_accuracy    : 85.963, Be

[2400/5000] Train loss: 0.00018, Valid loss: 0.27665, Elapsed_time: 399.91334
Current_accuracy : 85.829, Current_norm_ED  : 0.97
Best_accuracy    : 86.029, Best_norm_ED     : 0.97
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
994c50a20                 | 994c50a20                 | 0.7688	True
1311y21d33                | 1311y21d33                | 0.9700	True
995c44a10                 | 995c44a10                 | 0.5646	True
983h17b06                 | 883h17b06                 | 0.8222	False
9102                      | 9102                      | 0.9703	True
--------------------------------------------------------------------------------
[2500/5000] Train loss: 0.00017, Valid loss: 0.27443, Elapsed_time: 416.30549
Current_accuracy : 85.829, Current_norm_ED  : 0.97
Best_accuracy    : 86.029, B

[3400/5000] Train loss: 0.00010, Valid loss: 0.28819, Elapsed_time: 566.47992
Current_accuracy : 86.029, Current_norm_ED  : 0.97
Best_accuracy    : 86.029, Best_norm_ED     : 0.97
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
8094t43j23                | 8094t43j23                | 0.5572	True
7504                      | 7504                      | 0.9927	True
995e32b53                 | 995e32b53                 | 0.9812	True
5801                      | 5801                      | 0.6465	True
983s17t54                 | 983s17t54                 | 0.9976	True
--------------------------------------------------------------------------------
[3500/5000] Train loss: 0.00010, Valid loss: 0.28569, Elapsed_time: 582.68120
Current_accuracy : 85.896, Current_norm_ED  : 0.97
Best_accuracy    : 86.029, Be

[4400/5000] Train loss: 0.00008, Valid loss: 0.29359, Elapsed_time: 729.38389
Current_accuracy : 85.896, Current_norm_ED  : 0.97
Best_accuracy    : 86.096, Best_norm_ED     : 0.97
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
9012n15b17                | 9012h15b17                | 0.3826	False
2703                      | 2703                      | 0.6626	True
5903                      | 5903                      | 0.9340	True
0902                      | 0902                      | 0.9691	True
8109b38a10                | 8109b38a10                | 0.5268	True
--------------------------------------------------------------------------------
[4500/5000] Train loss: 0.00007, Valid loss: 0.29191, Elapsed_time: 745.68314
Current_accuracy : 86.029, Current_norm_ED  : 0.97
Best_accuracy    : 86.096, B

# 2. None-VGG-None-CTC.pth

In [9]:
args = argparse.Namespace(
    train_data = "../datasets/forTrainingData/train",
    valid_data = "../datasets/forTrainingData/val",
    select_data = "01-02-03",
    batch_ratio = "0.3-0.3-0.4",
    Transformation = "None",
    FeatureExtraction = "VGG",
    SequenceModeling = "None",
    Prediction = "CTC",
    saved_model = "models/None-VGG-None-CTC.pth",
    FT = True,
    manualSeed = 213,
    
    # Add arg for Training
    num_gpu = None,
    exp_name = None,
    workers = 0,
    batch_size = 192,
    data_filtering_off = None,
    imgH = 32,
    imgW = 100,
    num_iter = 5000,
    valInterval = 100, 
    adam = None,
    lr = 1,
    beta1 = 0.9,
    rho = 0.95,
    eps = 1e-8,
    grad_clip = 5,
    baiduCTC = None,
    total_data_usage_ratio = "1.0",
    batch_max_length = 25,
    rgb = None,
    sensitive = None,
    PAD = None,
    num_fiducial = 20,
    input_channel = 1,
    output_channel = 512,
    hidden_size = 256,
    character = '0123456789abcdefghijklmnopqrstuvwxyz'
    
)

In [10]:
opt = args

if not opt.exp_name:
    opt.exp_name = f'{opt.Transformation}-{opt.FeatureExtraction}-{opt.SequenceModeling}-{opt.Prediction}'
    opt.exp_name += f'-Seed{opt.manualSeed}'
    # print(opt.exp_name)

os.makedirs(f'./saved_models/{opt.exp_name}', exist_ok=True)

""" vocab / character number configuration """
if opt.sensitive:
    # opt.character += 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    opt.character = string.printable[:-6]  # same with ASTER setting (use 94 char).

""" Seed and GPU setting """
# print("Random Seed: ", opt.manualSeed)
random.seed(opt.manualSeed)
np.random.seed(opt.manualSeed)
torch.manual_seed(opt.manualSeed)
torch.cuda.manual_seed(opt.manualSeed)

cudnn.benchmark = True
cudnn.deterministic = True
opt.num_gpu = torch.cuda.device_count()
# print('device count', opt.num_gpu)
if opt.num_gpu > 1:
    print('------ Use multi-GPU setting ------')
    print('if you stuck too long time with multi-GPU setting, try to set --workers 0')
    # check multi-GPU issue https://github.com/clovaai/deep-text-recognition-benchmark/issues/1
    opt.workers = opt.workers * opt.num_gpu
    opt.batch_size = opt.batch_size * opt.num_gpu

    """ previous version
    print('To equlize batch stats to 1-GPU setting, the batch_size is multiplied with num_gpu and multiplied batch_size is ', opt.batch_size)
    opt.batch_size = opt.batch_size * opt.num_gpu
    print('To equalize the number of epochs to 1-GPU setting, num_iter is divided with num_gpu by default.')
    If you dont care about it, just commnet out these line.)
    opt.num_iter = int(opt.num_iter / opt.num_gpu)
    """

train(opt)

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: ../datasets/forTrainingData/train
opt.select_data: ['01', '02', '03']
opt.batch_ratio: ['0.3', '0.3', '0.4']
--------------------------------------------------------------------------------
dataset_root:    ../datasets/forTrainingData/train	 dataset: 01
sub-directory:	/01	 num samples: 2989
num total samples of 01: 2989 x 1.0 (total_data_usage_ratio) = 2989
num samples of 01 per batch: 192 x 0.3 (batch_ratio) = 58
--------------------------------------------------------------------------------
dataset_root:    ../datasets/forTrainingData/train	 dataset: 02
sub-directory:	/02	 num samples: 2872
num total samples of 02: 2872 x 1.0 (total_data_usage_ratio) = 2872
num samples of 02 per batch: 192 x 0.3 (batch_ratio) = 58
-----------------------------------

[500/5000] Train loss: 0.02552, Valid loss: 0.21908, Elapsed_time: 71.12743
Current_accuracy : 82.086, Current_norm_ED  : 0.97
Best_accuracy    : 82.086, Best_norm_ED     : 0.97
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
995b11t51                 | 995b11f51                 | 0.1975	False
2502                      | 2502                      | 0.9875	True
8093s11r05                | 8093s11r05                | 0.8339	True
983d31f26                 | 983d1f26                  | 0.2858	False
8702                      | 8302                      | 0.7168	False
--------------------------------------------------------------------------------
[600/5000] Train loss: 0.01785, Valid loss: 0.22713, Elapsed_time: 85.38927
Current_accuracy : 82.553, Current_norm_ED  : 0.97
Best_accuracy    : 82.553, Bes

[1500/5000] Train loss: 0.00156, Valid loss: 0.27209, Elapsed_time: 214.03362
Current_accuracy : 84.024, Current_norm_ED  : 0.97
Best_accuracy    : 84.024, Best_norm_ED     : 0.97
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
2604                      | 2604                      | 0.7389	True
8108b61307                | 8108b61307                | 0.1178	True
4001                      | 4001                      | 0.9929	True
994c32a14                 | 994c32a14                 | 0.4100	True
983e33t54                 | 983e33t54                 | 0.5438	True
--------------------------------------------------------------------------------
[1600/5000] Train loss: 0.00178, Valid loss: 0.28054, Elapsed_time: 228.24927
Current_accuracy : 84.225, Current_norm_ED  : 0.97
Best_accuracy    : 84.225, Be

[2500/5000] Train loss: 0.00033, Valid loss: 0.31702, Elapsed_time: 356.25672
Current_accuracy : 84.024, Current_norm_ED  : 0.97
Best_accuracy    : 84.693, Best_norm_ED     : 0.97
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
b109s23f42                | 8109s23f42                | 0.9490	False
8109h13b19                | 8109mh13b19               | 0.5143	False
7002                      | 7002                      | 0.9621	True
3101                      | 3101                      | 0.9170	True
8094h13b01                | 8094h13b01                | 0.3757	True
--------------------------------------------------------------------------------
[2600/5000] Train loss: 0.00035, Valid loss: 0.31933, Elapsed_time: 371.13118
Current_accuracy : 84.225, Current_norm_ED  : 0.97
Best_accuracy    : 84.693, 

[3500/5000] Train loss: 0.00051, Valid loss: 0.30610, Elapsed_time: 500.78313
Current_accuracy : 84.358, Current_norm_ED  : 0.97
Best_accuracy    : 84.693, Best_norm_ED     : 0.97
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
9901                      | 9901                      | 1.0000	True
8094t15j69                | 8094tf15j69               | 0.6467	False
8094b21a05                | 8094d21a05                | 0.9124	False
8109e42m70                | 8109e42m70                | 0.9945	True
8093f12327                | 8093f12327                | 0.7030	True
--------------------------------------------------------------------------------
[3600/5000] Train loss: 0.00026, Valid loss: 0.31750, Elapsed_time: 514.98048
Current_accuracy : 84.626, Current_norm_ED  : 0.97
Best_accuracy    : 84.693, 

[4500/5000] Train loss: 0.00011, Valid loss: 0.34829, Elapsed_time: 645.16328
Current_accuracy : 84.492, Current_norm_ED  : 0.97
Best_accuracy    : 84.693, Best_norm_ED     : 0.97
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
5002                      | 5002                      | 0.9984	True
983s17t54                 | 983s17t54                 | 0.3360	True
8109t21b16                | 8109t21b16                | 0.9997	True
7503                      | 7503                      | 0.5417	True
8603                      | 8603                      | 1.0000	True
--------------------------------------------------------------------------------
[4600/5000] Train loss: 0.00012, Valid loss: 0.34752, Elapsed_time: 659.32121
Current_accuracy : 84.559, Current_norm_ED  : 0.97
Best_accuracy    : 84.693, Be

# 3. TPS-ResNet-BiLSTM-Attn.pth

In [11]:
args = argparse.Namespace(
    train_data = "../datasets/forTrainingData/train",
    valid_data = "../datasets/forTrainingData/val",
    select_data = "01-02-03",
    batch_ratio = "0.3-0.3-0.4",
    Transformation = "TPS",
    FeatureExtraction = "ResNet",
    SequenceModeling = "BiLSTM",
    Prediction = "Attn",
    saved_model = "models/TPS-ResNet-BiLSTM-Attn.pth",
    FT = True,
    manualSeed = 213,
    
    # Add arg for Training
    num_gpu = None,
    exp_name = None,
    workers = 0,
    batch_size = 192,
    data_filtering_off = None,
    imgH = 32,
    imgW = 100,
    num_iter = 5000,
    valInterval = 100, 
    adam = None,
    lr = 1,
    beta1 = 0.9,
    rho = 0.95,
    eps = 1e-8,
    grad_clip = 5,
    baiduCTC = None,
    total_data_usage_ratio = "1.0",
    batch_max_length = 25,
    rgb = None,
    sensitive = None,
    PAD = None,
    num_fiducial = 20,
    input_channel = 1,
    output_channel = 512,
    hidden_size = 256,
    character = '0123456789abcdefghijklmnopqrstuvwxyz'
    
)

In [12]:
opt = args

if not opt.exp_name:
    opt.exp_name = f'{opt.Transformation}-{opt.FeatureExtraction}-{opt.SequenceModeling}-{opt.Prediction}'
    opt.exp_name += f'-Seed{opt.manualSeed}'
    # print(opt.exp_name)

os.makedirs(f'./saved_models/{opt.exp_name}', exist_ok=True)

""" vocab / character number configuration """
if opt.sensitive:
    # opt.character += 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    opt.character = string.printable[:-6]  # same with ASTER setting (use 94 char).

""" Seed and GPU setting """
# print("Random Seed: ", opt.manualSeed)
random.seed(opt.manualSeed)
np.random.seed(opt.manualSeed)
torch.manual_seed(opt.manualSeed)
torch.cuda.manual_seed(opt.manualSeed)

cudnn.benchmark = True
cudnn.deterministic = True
opt.num_gpu = torch.cuda.device_count()
# print('device count', opt.num_gpu)
if opt.num_gpu > 1:
    print('------ Use multi-GPU setting ------')
    print('if you stuck too long time with multi-GPU setting, try to set --workers 0')
    # check multi-GPU issue https://github.com/clovaai/deep-text-recognition-benchmark/issues/1
    opt.workers = opt.workers * opt.num_gpu
    opt.batch_size = opt.batch_size * opt.num_gpu

    """ previous version
    print('To equlize batch stats to 1-GPU setting, the batch_size is multiplied with num_gpu and multiplied batch_size is ', opt.batch_size)
    opt.batch_size = opt.batch_size * opt.num_gpu
    print('To equalize the number of epochs to 1-GPU setting, num_iter is divided with num_gpu by default.')
    If you dont care about it, just commnet out these line.)
    opt.num_iter = int(opt.num_iter / opt.num_gpu)
    """

train(opt)

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: ../datasets/forTrainingData/train
opt.select_data: ['01', '02', '03']
opt.batch_ratio: ['0.3', '0.3', '0.4']
--------------------------------------------------------------------------------
dataset_root:    ../datasets/forTrainingData/train	 dataset: 01
sub-directory:	/01	 num samples: 2989
num total samples of 01: 2989 x 1.0 (total_data_usage_ratio) = 2989
num samples of 01 per batch: 192 x 0.3 (batch_ratio) = 58
--------------------------------------------------------------------------------
dataset_root:    ../datasets/forTrainingData/train	 dataset: 02
sub-directory:	/02	 num samples: 2872
num total samples of 02: 2872 x 1.0 (total_data_usage_ratio) = 2872
num samples of 02 per batch: 192 x 0.3 (batch_ratio) = 58
-----------------------------------

[1/5000] Train loss: 3.32478, Valid loss: 2.45784, Elapsed_time: 4.41351
Current_accuracy : 13.703, Current_norm_ED  : 0.47
Best_accuracy    : 13.703, Best_norm_ED     : 0.47
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
995c44f29                 | 9934442299                | 0.0054	False
8108b61307                | 8038611091111111110000000 | 0.0000	False
8094h15b11                | 2444151111111144111111111 | 0.0000	False
8094h13b08                | 2440000000000000000000000 | 0.0000	False
5301                      | 5301                      | 0.4366	True
--------------------------------------------------------------------------------
[100/5000] Train loss: 0.21018, Valid loss: 0.10737, Elapsed_time: 99.48005
Current_accuracy : 89.840, Current_norm_ED  : 0.98
Best_accuracy    : 89.840, Best_

[1000/5000] Train loss: 0.00310, Valid loss: 0.11958, Elapsed_time: 882.26054
Current_accuracy : 91.845, Current_norm_ED  : 0.98
Best_accuracy    : 91.845, Best_norm_ED     : 0.98
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
8601                      | 8601                      | 0.9997	True
5002                      | 5002                      | 1.0000	True
9604                      | 9604                      | 0.9944	True
8094b56r03                | 8094b58r03                | 0.4744	False
4902                      | 4902                      | 0.9997	True
--------------------------------------------------------------------------------
[1100/5000] Train loss: 0.00292, Valid loss: 0.13087, Elapsed_time: 969.61107
Current_accuracy : 91.778, Current_norm_ED  : 0.98
Best_accuracy    : 91.845, B

[2000/5000] Train loss: 0.00029, Valid loss: 0.16191, Elapsed_time: 1756.01051
Current_accuracy : 91.243, Current_norm_ED  : 0.98
Best_accuracy    : 91.845, Best_norm_ED     : 0.98
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
8604                      | 8604                      | 0.9999	True
8094h15b04                | 8094h15b04                | 0.9996	True
5001                      | 5001                      | 0.9997	True
995e33a12                 | 995e33a12                 | 0.9999	True
8094d11a11                | 8094d11a11                | 0.9998	True
--------------------------------------------------------------------------------
[2100/5000] Train loss: 0.00028, Valid loss: 0.15713, Elapsed_time: 1842.29781
Current_accuracy : 91.110, Current_norm_ED  : 0.98
Best_accuracy    : 91.845, 

[3000/5000] Train loss: 0.00045, Valid loss: 0.14442, Elapsed_time: 2625.10650
Current_accuracy : 91.377, Current_norm_ED  : 0.98
Best_accuracy    : 91.845, Best_norm_ED     : 0.98
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
8108b11h41                | 8108d11h41                | 0.9962	False
995b17r09                 | 995b17r09                 | 0.9996	True
6803                      | 6803                      | 0.9719	True
8093g12a33                | 8093f12a33                | 0.8211	False
2604                      | 2604                      | 0.9907	True
--------------------------------------------------------------------------------
[3100/5000] Train loss: 0.00024, Valid loss: 0.14584, Elapsed_time: 2711.89866
Current_accuracy : 91.711, Current_norm_ED  : 0.98
Best_accuracy    : 91.845

[4000/5000] Train loss: 0.00006, Valid loss: 0.15842, Elapsed_time: 3493.19422
Current_accuracy : 91.979, Current_norm_ED  : 0.99
Best_accuracy    : 92.045, Best_norm_ED     : 0.99
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
8094h22f27                | 8094h22f27                | 0.9997	True
4602                      | 4602                      | 1.0000	True
983h13b14                 | 983h13b14                 | 0.9999	True
983e33t54                 | 983e33t54                 | 0.9997	True
995b17a17                 | 995b17a17                 | 0.9998	True
--------------------------------------------------------------------------------
[4100/5000] Train loss: 0.00064, Valid loss: 0.15675, Elapsed_time: 3579.94009
Current_accuracy : 91.912, Current_norm_ED  : 0.99
Best_accuracy    : 92.045, 

[5000/5000] Train loss: 0.00015, Valid loss: 0.15629, Elapsed_time: 4361.96695
Current_accuracy : 91.578, Current_norm_ED  : 0.99
Best_accuracy    : 92.045, Best_norm_ED     : 0.99
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
8108b64312                | 8108b64312                | 0.9991	True
8094d21a05                | 8094d21a05                | 0.9998	True
8094t43j23                | 8094t43j23                | 0.9996	True
982d43h40                 | 982d43h40                 | 0.9997	True
995b11t51                 | 995b11f51                 | 0.9938	False
--------------------------------------------------------------------------------
end the training


# 4. TPS-ResNet-BiLSTM-CTC.pth

In [13]:
args = argparse.Namespace(
    train_data = "../datasets/forTrainingData/train",
    valid_data = "../datasets/forTrainingData/val",
    select_data = "01-02-03",
    batch_ratio = "0.3-0.3-0.4",
    Transformation = "TPS",
    FeatureExtraction = "ResNet",
    SequenceModeling = "BiLSTM",
    Prediction = "CTC",
    saved_model = "models/TPS-ResNet-BiLSTM-CTC.pth",
    FT = True,
    manualSeed = 213,
    
    # Add arg for Training
    num_gpu = None,
    exp_name = None,
    workers = 0,
    batch_size = 192,
    data_filtering_off = None,
    imgH = 32,
    imgW = 100,
    num_iter = 5000,
    valInterval = 100, 
    adam = None,
    lr = 1,
    beta1 = 0.9,
    rho = 0.95,
    eps = 1e-8,
    grad_clip = 5,
    baiduCTC = None,
    total_data_usage_ratio = "1.0",
    batch_max_length = 25,
    rgb = None,
    sensitive = None,
    PAD = None,
    num_fiducial = 20,
    input_channel = 1,
    output_channel = 512,
    hidden_size = 256,
    character = '0123456789abcdefghijklmnopqrstuvwxyz'
    
)

In [14]:
opt = args

if not opt.exp_name:
    opt.exp_name = f'{opt.Transformation}-{opt.FeatureExtraction}-{opt.SequenceModeling}-{opt.Prediction}'
    opt.exp_name += f'-Seed{opt.manualSeed}'
    # print(opt.exp_name)

os.makedirs(f'./saved_models/{opt.exp_name}', exist_ok=True)

""" vocab / character number configuration """
if opt.sensitive:
    # opt.character += 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    opt.character = string.printable[:-6]  # same with ASTER setting (use 94 char).

""" Seed and GPU setting """
# print("Random Seed: ", opt.manualSeed)
random.seed(opt.manualSeed)
np.random.seed(opt.manualSeed)
torch.manual_seed(opt.manualSeed)
torch.cuda.manual_seed(opt.manualSeed)

cudnn.benchmark = True
cudnn.deterministic = True
opt.num_gpu = torch.cuda.device_count()
# print('device count', opt.num_gpu)
if opt.num_gpu > 1:
    print('------ Use multi-GPU setting ------')
    print('if you stuck too long time with multi-GPU setting, try to set --workers 0')
    # check multi-GPU issue https://github.com/clovaai/deep-text-recognition-benchmark/issues/1
    opt.workers = opt.workers * opt.num_gpu
    opt.batch_size = opt.batch_size * opt.num_gpu

    """ previous version
    print('To equlize batch stats to 1-GPU setting, the batch_size is multiplied with num_gpu and multiplied batch_size is ', opt.batch_size)
    opt.batch_size = opt.batch_size * opt.num_gpu
    print('To equalize the number of epochs to 1-GPU setting, num_iter is divided with num_gpu by default.')
    If you dont care about it, just commnet out these line.)
    opt.num_iter = int(opt.num_iter / opt.num_gpu)
    """

train(opt)

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: ../datasets/forTrainingData/train
opt.select_data: ['01', '02', '03']
opt.batch_ratio: ['0.3', '0.3', '0.4']
--------------------------------------------------------------------------------
dataset_root:    ../datasets/forTrainingData/train	 dataset: 01
sub-directory:	/01	 num samples: 2989
num total samples of 01: 2989 x 1.0 (total_data_usage_ratio) = 2989
num samples of 01 per batch: 192 x 0.3 (batch_ratio) = 58
--------------------------------------------------------------------------------
dataset_root:    ../datasets/forTrainingData/train	 dataset: 02
sub-directory:	/02	 num samples: 2872
num total samples of 02: 2872 x 1.0 (total_data_usage_ratio) = 2872
num samples of 02 per batch: 192 x 0.3 (batch_ratio) = 58
-----------------------------------

[1/5000] Train loss: 6.14577, Valid loss: 3.22677, Elapsed_time: 0.88189
Current_accuracy : 7.487, Current_norm_ED  : 0.38
Best_accuracy    : 7.487, Best_norm_ED     : 0.38
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
3603                      | xo3                       | 0.0261	False
8094b20617                | si2269                    | 0.0000	False
4001                      | gooj                      | 0.0086	False
8109f11a32                | bogfuaie                  | 0.0000	False
3701                      | 3701                      | 0.7318	True
--------------------------------------------------------------------------------
[100/5000] Train loss: 0.35205, Valid loss: 0.15922, Elapsed_time: 84.52140
Current_accuracy : 83.757, Current_norm_ED  : 0.97
Best_accuracy    : 83.757, Best_no

[1000/5000] Train loss: 0.00760, Valid loss: 0.14326, Elapsed_time: 842.46669
Current_accuracy : 90.241, Current_norm_ED  : 0.98
Best_accuracy    : 90.575, Best_norm_ED     : 0.98
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
4101                      | 4101                      | 0.9954	True
995e36t39                 | 995e36t39                 | 0.9625	True
6103                      | 6103                      | 0.9751	True
983e42f22                 | 983e41f22                 | 0.8977	False
2703                      | 2703                      | 0.9997	True
--------------------------------------------------------------------------------
[1100/5000] Train loss: 0.00636, Valid loss: 0.14116, Elapsed_time: 926.25080
Current_accuracy : 90.575, Current_norm_ED  : 0.98
Best_accuracy    : 90.575, B

[2000/5000] Train loss: 0.00126, Valid loss: 0.16687, Elapsed_time: 1680.93719
Current_accuracy : 90.241, Current_norm_ED  : 0.98
Best_accuracy    : 90.842, Best_norm_ED     : 0.98
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
9012e31a31                | 9012e31a31                | 0.5197	True
4504                      | 4604                      | 0.5520	False
8109b42a11                | 8109b42a11                | 0.4022	True
2803                      | 2803                      | 0.9997	True
3001                      | 3001                      | 0.9995	True
--------------------------------------------------------------------------------
[2100/5000] Train loss: 0.00130, Valid loss: 0.16972, Elapsed_time: 1764.83123
Current_accuracy : 90.642, Current_norm_ED  : 0.98
Best_accuracy    : 90.842,

[3000/5000] Train loss: 0.00080, Valid loss: 0.17865, Elapsed_time: 2518.21416
Current_accuracy : 90.642, Current_norm_ED  : 0.98
Best_accuracy    : 90.842, Best_norm_ED     : 0.98
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
8108b61d35                | 8108b61d35                | 0.4704	True
983h17b07                 | 983h17b07                 | 0.6453	True
8001                      | 6001                      | 0.9995	False
5203                      | 5203                      | 0.9956	True
983e41f22                 | 983e41f22                 | 0.9814	True
--------------------------------------------------------------------------------
[3100/5000] Train loss: 0.00069, Valid loss: 0.18347, Elapsed_time: 2601.93974
Current_accuracy : 90.842, Current_norm_ED  : 0.98
Best_accuracy    : 90.842,

[4000/5000] Train loss: 0.00045, Valid loss: 0.18407, Elapsed_time: 3355.69084
Current_accuracy : 91.110, Current_norm_ED  : 0.98
Best_accuracy    : 91.110, Best_norm_ED     : 0.98
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
7001                      | 7001                      | 0.9196	True
983e22b12                 | 983e22b12                 | 0.9946	True
4402                      | 4102                      | 0.9675	False
9012e32a19                | 9012e52a19                | 0.8898	False
8094t43j23                | 8094t43j23                | 0.7796	True
--------------------------------------------------------------------------------
[4100/5000] Train loss: 0.00031, Valid loss: 0.19738, Elapsed_time: 3439.61374
Current_accuracy : 90.842, Current_norm_ED  : 0.98
Best_accuracy    : 91.110

[5000/5000] Train loss: 0.00029, Valid loss: 0.20560, Elapsed_time: 4192.35881
Current_accuracy : 90.441, Current_norm_ED  : 0.98
Best_accuracy    : 91.243, Best_norm_ED     : 0.98
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
995b14f39                 | 995b14f39                 | 0.8425	True
9012f41a09                | 9012f41a09                | 0.9909	True
5603                      | 5603                      | 0.9931	True
8094s13b57                | 8094s13b57                | 0.7771	True
0703                      | 0703                      | 0.9939	True
--------------------------------------------------------------------------------
end the training


# 5. None-ResNet-None-CTC.pth

In [15]:
args = argparse.Namespace(
    train_data = "../datasets/forTrainingData/train",
    valid_data = "../datasets/forTrainingData/val",
    select_data = "01-02-03",
    batch_ratio = "0.3-0.3-0.4",
    Transformation = "None",
    FeatureExtraction = "ResNet",
    SequenceModeling = "None",
    Prediction = "CTC",
    saved_model = "models/None-ResNet-None-CTC.pth",
    FT = True,
    manualSeed = 213,
    
    # Add arg for Training
    num_gpu = None,
    exp_name = None,
    workers = 0,
    batch_size = 192,
    data_filtering_off = None,
    imgH = 32,
    imgW = 100,
    num_iter = 5000,
    valInterval = 100, 
    adam = None,
    lr = 1,
    beta1 = 0.9,
    rho = 0.95,
    eps = 1e-8,
    grad_clip = 5,
    baiduCTC = None,
    total_data_usage_ratio = "1.0",
    batch_max_length = 25,
    rgb = None,
    sensitive = None,
    PAD = None,
    num_fiducial = 20,
    input_channel = 1,
    output_channel = 512,
    hidden_size = 256,
    character = '0123456789abcdefghijklmnopqrstuvwxyz'
    
)

In [16]:
opt = args

if not opt.exp_name:
    opt.exp_name = f'{opt.Transformation}-{opt.FeatureExtraction}-{opt.SequenceModeling}-{opt.Prediction}'
    opt.exp_name += f'-Seed{opt.manualSeed}'
    # print(opt.exp_name)

os.makedirs(f'./saved_models/{opt.exp_name}', exist_ok=True)

""" vocab / character number configuration """
if opt.sensitive:
    # opt.character += 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    opt.character = string.printable[:-6]  # same with ASTER setting (use 94 char).

""" Seed and GPU setting """
# print("Random Seed: ", opt.manualSeed)
random.seed(opt.manualSeed)
np.random.seed(opt.manualSeed)
torch.manual_seed(opt.manualSeed)
torch.cuda.manual_seed(opt.manualSeed)

cudnn.benchmark = True
cudnn.deterministic = True
opt.num_gpu = torch.cuda.device_count()
# print('device count', opt.num_gpu)
if opt.num_gpu > 1:
    print('------ Use multi-GPU setting ------')
    print('if you stuck too long time with multi-GPU setting, try to set --workers 0')
    # check multi-GPU issue https://github.com/clovaai/deep-text-recognition-benchmark/issues/1
    opt.workers = opt.workers * opt.num_gpu
    opt.batch_size = opt.batch_size * opt.num_gpu

    """ previous version
    print('To equlize batch stats to 1-GPU setting, the batch_size is multiplied with num_gpu and multiplied batch_size is ', opt.batch_size)
    opt.batch_size = opt.batch_size * opt.num_gpu
    print('To equalize the number of epochs to 1-GPU setting, num_iter is divided with num_gpu by default.')
    If you dont care about it, just commnet out these line.)
    opt.num_iter = int(opt.num_iter / opt.num_gpu)
    """

train(opt)

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: ../datasets/forTrainingData/train
opt.select_data: ['01', '02', '03']
opt.batch_ratio: ['0.3', '0.3', '0.4']
--------------------------------------------------------------------------------
dataset_root:    ../datasets/forTrainingData/train	 dataset: 01
sub-directory:	/01	 num samples: 2989
num total samples of 01: 2989 x 1.0 (total_data_usage_ratio) = 2989
num samples of 01 per batch: 192 x 0.3 (batch_ratio) = 58
--------------------------------------------------------------------------------
dataset_root:    ../datasets/forTrainingData/train	 dataset: 02
sub-directory:	/02	 num samples: 2872
num total samples of 02: 2872 x 1.0 (total_data_usage_ratio) = 2872
num samples of 02 per batch: 192 x 0.3 (batch_ratio) = 58
-----------------------------------

[1/5000] Train loss: 17.70298, Valid loss: 4.86960, Elapsed_time: 0.78249
Current_accuracy : 3.409, Current_norm_ED  : 0.22
Best_accuracy    : 3.409, Best_norm_ED     : 0.22
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
8108b38m51                | s0bbsm                    | 0.0002	False
6501                      | 6501                      | 0.2493	True
995b13g49                 | asbeghd                   | 0.0007	False
8108b23d2                 | s08b2dd                   | 0.0019	False
995m31b07                 | tcoikon                   | 0.0006	False
--------------------------------------------------------------------------------
[100/5000] Train loss: 1.01397, Valid loss: 0.23527, Elapsed_time: 75.75139
Current_accuracy : 74.332, Current_norm_ED  : 0.95
Best_accuracy    : 74.332, Best_n

[1000/5000] Train loss: 0.00457, Valid loss: 0.20620, Elapsed_time: 755.05522
Current_accuracy : 86.832, Current_norm_ED  : 0.97
Best_accuracy    : 87.366, Best_norm_ED     : 0.97
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
5902                      | 5902                      | 0.7055	True
8601                      | 8601                      | 1.0000	True
994e43t09                 | 994e43t09                 | 0.7203	True
9301                      | 9701                      | 0.6904	False
8093a21307                | 8093a21307                | 0.6841	True
--------------------------------------------------------------------------------
[1100/5000] Train loss: 0.00329, Valid loss: 0.21207, Elapsed_time: 830.08114
Current_accuracy : 86.832, Current_norm_ED  : 0.97
Best_accuracy    : 87.366, B

[2000/5000] Train loss: 0.00058, Valid loss: 0.25304, Elapsed_time: 1508.65237
Current_accuracy : 87.299, Current_norm_ED  : 0.97
Best_accuracy    : 87.366, Best_norm_ED     : 0.97
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
8094e12r07                | 8094e12r07                | 0.9999	True
9301                      | 9301                      | 0.9870	True
8094h21f39                | 8094h21f39                | 0.6683	True
7002                      | 7002                      | 0.9995	True
995b17r09                 | 995b17r09                 | 0.8590	True
--------------------------------------------------------------------------------
[2100/5000] Train loss: 0.00045, Valid loss: 0.26048, Elapsed_time: 1583.75995
Current_accuracy : 87.166, Current_norm_ED  : 0.97
Best_accuracy    : 87.366, 

[3000/5000] Train loss: 0.00017, Valid loss: 0.27735, Elapsed_time: 2260.95931
Current_accuracy : 87.299, Current_norm_ED  : 0.97
Best_accuracy    : 87.366, Best_norm_ED     : 0.97
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
8601                      | 8601                      | 0.9999	True
9604                      | 9604                      | 0.9998	True
995c44a09                 | 995c44a09                 | 0.6472	True
5801                      | 5801                      | 0.6381	True
5903                      | 5903                      | 0.9998	True
--------------------------------------------------------------------------------
[3100/5000] Train loss: 0.00018, Valid loss: 0.27905, Elapsed_time: 2336.27106
Current_accuracy : 87.299, Current_norm_ED  : 0.97
Best_accuracy    : 87.366, 

[4000/5000] Train loss: 0.00060, Valid loss: 0.28590, Elapsed_time: 3013.04254
Current_accuracy : 87.032, Current_norm_ED  : 0.97
Best_accuracy    : 87.366, Best_norm_ED     : 0.97
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
1202                      | 1202                      | 0.6772	True
983n91b07                 | 983h91b07                 | 0.9973	False
6501                      | 6501                      | 0.9709	True
8108b61307                | 8108b61307                | 0.6871	True
8109b42g26                | 8109b42g26                | 0.8236	True
--------------------------------------------------------------------------------
[4100/5000] Train loss: 0.00016, Valid loss: 0.28917, Elapsed_time: 3088.52119
Current_accuracy : 87.032, Current_norm_ED  : 0.97
Best_accuracy    : 87.366,

[5000/5000] Train loss: 0.00012, Valid loss: 0.29790, Elapsed_time: 3766.20593
Current_accuracy : 86.765, Current_norm_ED  : 0.97
Best_accuracy    : 87.433, Best_norm_ED     : 0.97
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
995e33a12                 | 995e33a12                 | 0.8834	True
0001                      | 0001                      | 1.0000	True
8108f41310                | 8108f41310                | 0.5938	True
995e36t39                 | 995e36t39                 | 0.9967	True
5801                      | 5801                      | 0.6714	True
--------------------------------------------------------------------------------
end the training
